In [1]:
%pip install openai


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()


client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [19]:
file_name = "./orders.csv"
assistant_name = "zensar-analyst"

file_list = client.files.list(purpose="assistants")
filenames = [x.filename for x in file_list.data]

if not file_name in filenames:
    file = client.files.create(
        file=open(file_name, "rb"),
        purpose="assistants"
    )
else:
    for f in file_list:
        if f.filename == file_name:
            file = client.files.retrieve(f.id)
            break

In [20]:
assistant_list = client.beta.assistants.list()
assistant_names = [x.name for x in assistant_list.data]
assistant_exists = True if assistant_name in assistant_names else False

In [21]:
assistant_exists

True

In [22]:
if not assistant_exists:
    assistant = client.beta.assistants.create(
        name = assistant_name,
        instructions="Your are a data analyst",
        model = "gpt-4o",
        tools = [
            {
                "type": "code_interpreter"
            }
        ],
        tool_resources = {
            "code_interpreter": {
                "file_ids": [file.id]
            }
        }
    )
else:
    for a in assistant_list:
        if a.name == assistant_name:
            assistant = client.beta.assistants.retrieve(a.id)
            break

In [23]:
prompt = """
using the csv file attached, display a pie graph for number of orders by customer type
"""

thread = client.beta.threads.create(
    messages = [
        {
            "role":"user",
            "content": prompt,
            "attachments": [
                {
                    "file_id": file.id,
                    "tools": [
                        {
                            "type":"code_interpreter"
                        }
                    ]
                }
            ]
        }
    ]
)

In [24]:
image_file_name = "my-image2.png"

In [29]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.


class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

In [30]:
with client.beta.threads.runs.stream(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = f"Create a downloadable file for the graph called {image_file_name}",
    event_handler = EventHandler()
) as stream:
    stream.until_done()


assistant > Feel free to request any further modifications or additional charts if needed!

In [31]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="""
        Create a downloadable file for the graph called {image_file_name}
    """
)

if run.status == "Completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)

    print(messages)
else:
    print(run.status)

completed


In [32]:
filelist = client.files.list(purpose="assistants_output")

image_list = [x for x in filelist.data if image_file_name in x.filename]

len(image_list)

0

In [13]:
id = image_list[0].id

In [16]:
image_data = client.files.content(id)
image_data_bytes = image_data.read()

with open(f"./{image_file_name}", "wb") as file:
    file.write(image_data_bytes)

![](./my-image.png)